In [ ]:
from mylibraries import *
X_train = pd.read_csv('X_train_scaled.csv')
X_test = pd.read_csv('X_test_scaled.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Získanie názvov stĺpcov z pôvodného X_train
column_names = X_train.columns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.model_selection import learning_curve


# Definícia modelu logistickej regresie
log_reg = LogisticRegression(C=10.0, max_iter=100, penalty='l2', solver='lbfgs')

# Trénovanie modelu logistickej regresie
log_reg.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_log_reg = log_reg.predict(X_test)

# Vyhodnotenie modelu
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print(f"Testovacia presnosť modelu logistickej regresie: {accuracy_log_reg:.4f}")

# Klasifikačná správa modelu logistickej regresie
print("Klasifikačná správa modelu logistickej regresie:")
print(classification_report(y_test, y_pred_log_reg))

# Matica zámen modelu logistickej regresie
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("Matica zámen modelu logistickej regresie:")
print(conf_matrix_log_reg)

# Predikcia pravdepodobností pre ROC a Precision-Recall krivky
y_prob_log_reg = log_reg.predict_proba(X_test)[:, 1]

# ROC krivka
fpr_log_reg, tpr_log_reg, _ = roc_curve(y_test, y_prob_log_reg)
plt.plot(fpr_log_reg, tpr_log_reg, label='ROC krivka (AUC = {:.2f})'.format(roc_auc_score(y_test, y_prob_log_reg)))
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Falošne pozitívna miera')
plt.ylabel('Pravdivo pozitívna miera')
plt.title('ROC Krivka pre logistickú regresiu')
plt.legend()
plt.show()

# Precision-Recall krivka
precision_log_reg, recall_log_reg, _ = precision_recall_curve(y_test, y_prob_log_reg)
average_precision_log_reg = average_precision_score(y_test, y_prob_log_reg)
plt.plot(recall_log_reg, precision_log_reg, label='Precision-Recall krivka (priemer = {:.2f})'.format(average_precision_log_reg))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Krivka pre logistickú regresiu')
plt.legend()
plt.show()

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(log_reg, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure()
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Trénovacia presnosť')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Validačná presnosť')

plt.xlabel('Počet trénovacích príkladov')
plt.ylabel('Presnosť')
plt.title('Learning Curve pre logistickú regresiu')
plt.legend(loc="best")
plt.show()

# Extrahovanie koeficientov modelu a identifikácia najdôležitejších atribútov
importance = abs(log_reg.coef_[0])
sorted_idx = importance.argsort()[::-1]
top_n = 10
sorted_idx_top_n = sorted_idx[:top_n]

# Vykreslenie 10 najdôležitejších atribútov
plt.barh(range(top_n), importance[sorted_idx_top_n])
plt.yticks(range(top_n), X_train.columns[sorted_idx_top_n])
plt.xlabel('Koeficienty')
plt.title('Dôležitosť atribútov (najdôležitejších 10)')
plt.show()

In [ ]:
# Výber len najdôležitejších atribútov pre trénovanie modelu
important_features = X_train.columns[sorted_idx_top_n]
X_train_important = X_train.iloc[:, sorted_idx_top_n]
X_test_important = X_test.iloc[:, sorted_idx_top_n]

# Trénovanie modelu logistickej regresie na najdôležitejších atribútoch
log_reg_important = LogisticRegression(C=10.0, max_iter=100, penalty='l2', solver='lbfgs')
log_reg_important.fit(X_train_important, y_train)

# Predikcia na testovacej množine
y_pred_log_reg_important = log_reg_important.predict(X_test_important)

# Vyhodnotenie modelu
accuracy_log_reg_important = accuracy_score(y_test, y_pred_log_reg_important)
print(f"Testovacia presnosť modelu logistickej regresie (najdôležitejšie atribúty): {accuracy_log_reg_important:.4f}")

# Klasifikačná správa modelu logistickej regresie
print("Klasifikačná správa modelu logistickej regresie (najdôležitejšie atribúty):")
print(classification_report(y_test, y_pred_log_reg_important))

# Matica zámen modelu logistickej regresie
print("Matica zámen modelu logistickej regresie (najdôležitejšie atribúty):")
print(confusion_matrix(y_test, y_pred_log_reg_important))

In [ ]:
from sklearn.ensemble import BaggingClassifier

# Definícia Bagging modelu s logistickou regresiou ako estimator
bagging_log_reg = BaggingClassifier(estimator=LogisticRegression(C=10.0, max_iter=100, penalty='l2', solver='lbfgs'), n_estimators=50, random_state=42, n_jobs=-1)

# Trénovanie Bagging modelu
bagging_log_reg.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_bagging_log_reg = bagging_log_reg.predict(X_test)

# Vyhodnotenie modelu
accuracy_bagging_log_reg = accuracy_score(y_test, y_pred_bagging_log_reg)
print(f"Testovacia presnosť Bagging modelu s logistickou regresiou: {accuracy_bagging_log_reg:.4f}")

# Klasifikačná správa Bagging modelu
print("Klasifikačná správa Bagging modelu s logistickou regresiou:")
print(classification_report(y_test, y_pred_bagging_log_reg))
print(confusion_matrix(y_test, y_pred_bagging_log_reg))
import seaborn as sns
import matplotlib.pyplot as plt